In [1]:
from dataset_io import *
from color_channels import *
import os
import cv2
import numpy as np
import random as ra
from tqdm import tqdm
from profiler import *
import random

In [2]:
def imread_resized(scene_name, filename, label_resolution):
    filepath=get_filepath(scene_name, filename)
    image = cv2.imread(filepath)
    sh=image.shape
    sh=(int(sh[1]/4), int(sh[0]/4))
    return cv2.resize(image, sh)

In [3]:
def convert_to_gradient_channels(img):
    channels=[]
    
    img = cv2.UMat(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY )
    channels.append(cv2.UMat(gray))
    
    gray = cv2.GaussianBlur(gray, (3,3), sigmaX=0, sigmaY=0)
    channels.append(cv2.UMat(gray))
    
    gray = cv2.equalizeHist(gray) #experimental
    gradX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3, scale=1 )
    gradY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3, scale=1 )
    mag, ori = cv2.cartToPolar(gradX, gradY, angleInDegrees=True)

    ori_ranges = [(0,60), (60,120), (120,180), (180,240), (240,300), (300,360)]
    oris=[cv2.inRange(ori,l,r) for (l,r) in ori_ranges]
    mags= [cv2.copyTo(mag, ori_x) for ori_x in oris]
    channels.extend([gray,mag]+mags)
    channels=[cv2.UMat.get(ch) for ch in channels]
    print(np.array(channels).shape)
    #channels=np.moveaxis(channels, 0, -1)
    return channels

In [4]:
img = imread_resized("wooden_folia_6", "0001.jpg", None)
cv2.imwrite("visualized/img.jpg", img)

channels=convert_to_gradient_channels(img)

for i in range(len(channels)):
    cv2.imwrite("visualized/channel_{}.jpg".format(i), channels[i])

(10, 180, 320)
